# Pokemon

## 1. Getting to know the data
## 2. Legendary pokemon prediction/Cross-Validation
## 3. Combat mode - up next


<img src=https://pics.me.me/whos-that-pokemon-maude-flanders-14530226.png>

# 1. Getting to know the data



In [ ]:
#import libraries,get used csv files and look at the data
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from subprocess import check_output
import warnings
#warnings.filterwarnings("ignore")

train = pd.read_csv('../input/combats.csv', encoding='ISO-8859-1',low_memory=False)
pokemon = pd.read_csv('../input/pokemon.csv', encoding='ISO-8859-1',low_memory=False)
test= pd.read_csv('../input/tests.csv', encoding='ISO-8859-1',low_memory=False)
#target=train["Winner"].values
#train= train.drop(["Winner"], axis=1)


In [ ]:
pokemon.tail(3)



In [ ]:
train.head(3) 

> Pokemon id's are in "#" column 

In [ ]:
pokemon['Type 1'].unique()




In [ ]:
fig, ax =plt.subplots()
fig.set_size_inches(16, 12)
types1 = sns.countplot(x = 'Type 1', data=pokemon, palette='Paired')
types1.set(xlabel='Type 1',ylabel='count', label= "Pokemon type 1")
plt.setp(types1.get_xticklabels(), rotation=90)
plt.show()

> Since this is an EDA for now, lets replace the pokemon id numbers with their names so that we can see who is used to winning and order them accordingly

In [ ]:
names_dict = dict(zip(pokemon["#"], pokemon["Name"]))
cols = ["Winner","First_pokemon","Second_pokemon"]
train[cols] = train[cols].replace(names_dict)
train.head(3)


> Let's also see a worldcloud of our strongest competitors

<img src=https://vignette.wikia.nocookie.net/pokemon/images/c/c6/Team_Rocket_trio_XY.png/revision/latest?cb=20131221014744>

In [ ]:
mask_mew = b'...'

In [ ]:
from wordcloud import WordCloud
from scipy.misc import imread
import base64
import codecs
import nltk

f = open("mew.png", "wb")
f.write(codecs.decode(mask_mew,'base64'))
f.close()
imgm = imread("mew.png")
maskm= imgm
winn=train["Winner"].dropna().values
stopwords = nltk.corpus.stopwords.words('english')
plt.figure(figsize=(14,12))
wc=WordCloud(relative_scaling = 1,max_words=500,
                      stopwords = stopwords, mask=maskm,background_color="black", max_font_size= 30).generate(" ".join(winn))

plt.imshow(wc.recolor(colormap= 'Spectral_r'), interpolation='bilinear')
plt.title("Winners", fontsize=16)
plt.axis("off")
plt.show()



#d = dict([tuple(x) for x in train.Winner.value_counts().reset_index().values])
#wordcloud = WordCloud().generate_from_frequencies(d)


In [ ]:
print(train["Winner"].unique())

> As we can see, Therian Forme and Maga for example are repeated in numerous types so those are the words that are also noticable in the wordcloud. Plotting 10 with most wins so we can see that Mewtwo has the most wins.


In [ ]:

win=train.Winner.value_counts()[:10]
#print(win)
fig, ax =plt.subplots()
fig.set_size_inches(16, 12)
sns.countplot(x='Winner',data = train,palette='Paired',order=win.index)
plt.show()


In [ ]:
print(pokemon.info())

# 2. Legendary pokemon prediction/Cross-Validation


> Can we spot a legendary pokemon only by looking at the data? Did some mapping, and sliced the pokemon dataset in two(600:200).

In [ ]:
pokemon["Type 2"]=pokemon["Type 2"].fillna(0)#fill NaNs in Type 2 
pokemon["Name"]=pokemon["Name"].fillna("Unknown")
#print(pokemon.isnull().sum())
#pokemon['Type 1'] = pokemon['Type 1'].apply(lambda x: str(x)) #
pokemon.Legendary = pokemon.Legendary.astype(int) #if by any chance i'll need an int not bool
mapping_type = {'Grass': 18, 'Fire': 1, 'Water': 2, 'Bug': 3, 'Normal': 4, 'Poison': 5, 'Electric': 6, 'Ground': 7
               , 'Fairy': 8, 'Fighting': 9, 'Psychic': 10, 'Rock': 11, 'Ghost': 12, 'Ice': 13, 'Dragon': 14, 'Dark': 15, 'Steel': 16, 'Flying': 17}
pokemon=pokemon.replace({'Type 1': mapping_type})
pokemon=pokemon.replace({'Type 2': mapping_type})
pokemon.head(10)
#pokemon['Type 1'] = pokemon['Type 1'].apply(lambda x: int(x))

In [ ]:
pokemon_train = pokemon.head(600)
pokemon_test = pokemon.iloc[600:]

In [ ]:
#correlation 
from matplotlib import cm
f,ax = plt.subplots(figsize=(16, 16))

sns.heatmap(pokemon_train.corr(), annot=True, linewidths=.5,ax=ax,cmap=cm.YlGnBu,)

> Removing unused columns and saving info about legendary pokemon. First one for training the dataset and the other one for comparing to our results.

In [ ]:
target_legendary=pokemon_train["Legendary"].values
pokemon_train= pokemon_train.drop(["Legendary","Name","#"], axis=1)

test_legendary=pokemon_test["Legendary"].values
pokemon_test= pokemon_test.drop(["Legendary","Name","#"], axis=1)

In [ ]:
##decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
clf=DecisionTreeClassifier()#
clf=clf.fit(pokemon_train,target_legendary)
#print(clf.feature_importances_) 
#print(clf.score(pokemon_train, target_legendary))
prediction_dt = clf.predict(pokemon_test)
accuracy = accuracy_score(test_legendary,prediction_dt)

#solution = pd.DataFrame(prediction_dt, columns = ["Legendary"])

#print(test_legendary)
print('Accuracy of the model: %.2f%%' % accuracy)
#accuracy
#r=[i for i, j in zip(test_legendary,prediction) if i == j]
#print(len(r)/200)

In [ ]:
#knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(pokemon_train,target_legendary)

prediction_knn = knn.predict(pokemon_test)
accuracy = accuracy_score(test_legendary,prediction_knn)
print('Accuracy of the model: %.2f%%' % accuracy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
forest=random_forest.fit(pokemon_train,target_legendary) 

prediction_rf=forest.predict(pokemon_test)
accuracy = accuracy_score(test_legendary,prediction_rf)
print('Accuracy of the model: %.2f%%' % accuracy)

In [ ]:
from xgboost import XGBClassifier

xg = XGBClassifier(max_depth=5, n_estimators=200, learning_rate=0.01)
xg.fit(pokemon_train,target_legendary)

prediction_xg = xg.predict(pokemon_test)

accuracy = accuracy_score(test_legendary,prediction_xg)
print('Accuracy of the model: %.2f%%' % accuracy)

> Now that we've looked at the training data a bit, and have looked at a couple of algorithms, we can return to the original problem - predicting winners in battle


# 3. Combat mode - up next
> to be continued


